# 다항회귀
> 선형회귀의 경우 설명변수와 종속변수의 관계를 선형관계로 해석하지만 현실 문제는 선형으로 해결이 불가능한 경우가 많습니다.  
선형모델의 구조적 한계를 보완하고자 **변수 간 영향력에 해당하는 새로운 변수를 생성**하여 방정식을 비선형으로 만들고 선형모델을 적용시킬 수 있습니다.  
2차원 예시 $x_1, x_2, x_3$를 $x_1^2, x_2^2, x_3^2, x_1x_2, x_1x_3, x_2x_3$으로 변환하여 변수를 확장.  
보통 2차원 또는 3차원까지 적용하며, 변수가 많아지므로 Ridge, Lasso 모델을 적용한다.
<img src="./image/47.png">

### 다항회귀 실습

In [1]:
import pandas as pd

In [2]:
# boston 데이터 확인
df = pd.read_csv('./data/boston.csv')

In [3]:
# 타겟데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [4]:
# 훈련셋과 테스트셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
# 전처리 모델 import
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

In [6]:
# 다항변수 제작함수 인스턴스화
poly = PolynomialFeatures(degree=2, include_bias=False)
'''
파라메터
degree=2 : 차수설정
include_bias=False : 상수항 제거(필수) b0 조합은 제거
'''

'\n파라메터\ndegree=2 : 차수설정\ninclude_bias=False : 상수항 제거(필수) b0 조합은 제거\n'

In [12]:
# 최대 표시가능한 컬럼 갯수 설정
pd.options.display.max_columns=200

In [14]:
# 전처리 모델의 학습
poly_X_train = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out())
# 설명변수 갯수, 설명변수 이름

In [16]:
# test 데이터 동일한 모델로 적용
poly_X_test = poly.transform(X_test)

In [36]:
# 모델 정의
poly_lasso = Lasso(alpha=0.5)

In [37]:
# 모델 학습
poly_lasso.fit(poly_X_train, y_train)

C:\ProgramData\miniconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.814e+03, tolerance: 3.112e+00
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.5)

In [38]:
# 모델 예측
poly_pred = poly_lasso.predict(poly_X_test)

C:\ProgramData\miniconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


In [39]:
# 모델 평가
from sklearn.metrics import r2_score, mean_squared_error
print(f'R2 score : {r2_score(y_test, poly_pred)}')
print(f'RMSE : {mean_squared_error(y_test, poly_pred, squared=False)}')

R2 score : 0.831006868958385
RMSE : 3.548549185740129


In [ ]:
'''lr
R2 score : 0.7112260057484925
RMSE : 4.638689926172827
rg a=1
R2 score : 0.7041586727559433
RMSE : 4.695109486461527
rg a=10
R2 score : 0.6954181695183624
RMSE : 4.763962116024238
poly a=1
R2 score : 0.8254946931329347
RMSE : 3.6059576547271877
'''

In [ ]:
# 모델 계수 확인


In [ ]:
# 계수 플로팅
plt.barh()

In [ ]:
# 예측성능이 최우선인 프로젝트에는 비벼볼만 한 모델
# 베타값의 해석, 모델에 따른 의사결정, 비즈니스 액션을 위한 모델로는 적합하지 않음